In [15]:
import json, re

In [ ]:
Color = ['yellow', 'aqua marine', 'blue', 'gray', 'pink', 'red', 'green']

In [16]:
def phone_format(phone_number):
    clean_phone_number = re.sub('[^0-9]+', '', phone_number)
    formatted_phone_number = re.sub("(\d)(?=(\d{3})+(?!\d))", r"\1-", "%d" % int(clean_phone_number[:-1])) + clean_phone_number[-1]
    return formatted_phone_number

In [26]:
phone_number = '(622)-720-0550'
clean_phone_number = re.sub('[^0-9]+', '', phone_number)
formatted_phone_number = re.sub("(\d)(?=(\d{3})+(?!\d))", r"\1-", "%d" % int(clean_phone_number[:-1])) + clean_phone_number[-1]

In [28]:
if len(clean_phone_number) ~= 9:
    return

10

In [14]:
f = open('data.in', 'r')
entries = {}
errors  = {}
lineIdx = 0
for line in f:
    x = line.split(",")
    
    # sanity check: completion of entries
    if len(x) > 5 or len(x) < 4:
        errors[lineInx] = 'invalid entry'
        continue
    
    #
    
    
    lineIdx += 1
    
    

{1, 4, 5}